# ACO - Vehicle Routing Problem with Capacity and Time Windows

In [1]:
from math import dist
from tqdm import tqdm 
import copy
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import random
import seaborn as sns

## Prepare data and graph, visualization of data points

In [2]:
class Graph:
    def __init__(self, x, y, n_depots) -> None:
        self.n = n = len(x)
        self.x = np.array(x, dtype=float)
        self.y = np.array(y, dtype=float)
        self.edges = np.zeros(shape=(n,n), dtype=float)

        for i in range(self.n):
            for j in range(self.n):
                self.edges[i,j] = dist([x[i], y[i]], [x[j], y[j]])

        xf = 10000
        # high distance from one point to itself
        for i in range(self.n):
            self.edges[i,i] = xf
            
        # high distance from one depot to another depot
        for i in range(n_depots):
            for j in range(n_depots):
                self.edges[i,j] = xf

In [3]:
def get_location_data():
    with open('rcdata/r112.txt', 'r', encoding = 'utf-8') as f:
        [f.readline() for i in range(4)]
        n_locations = 100
        max_vehicles, max_capacity = [int(x) for x in f.readline().split()]
        data = []
        [f.readline() for i in range(4)]
        for i in range(n_locations + 1):
            data.append([float(x) for x in f.readline().split()[1:]])

    # uncomment to see plot
    # data = np.array(data)
    # dataT = data.T
    # a = pd.DataFrame({'x':dataT[0], 'y':dataT[1] , 'd':dataT[2]})
    # sns.scatterplot(data=a, x='x', y='y', size='d', palette="deep")
    return data, n_locations, max_vehicles, max_capacity

In [4]:
def insert_duplicate_depots(depots_to_add):
    data, n_locations, max_vehicles, max_capacity = get_location_data()
    data = np.append(np.repeat([data[0]], depots_to_add, axis=0), data, axis=0)
    return data, n_locations, max_vehicles, max_capacity

In [5]:
def create_graph(n_vehicles):
    n_depots = n_vehicles + 1 # IMPORTANT
    data, n_locations, max_vehicles, max_capacity = insert_duplicate_depots(n_depots - 1)
    dataT = data.T
    graph = Graph(dataT[0], dataT[1], n_depots)
    return data, graph, n_locations, n_depots, max_vehicles, max_capacity 

In [6]:
n_vehicles = 10
data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)

In [7]:
print(data.shape, graph.edges.shape, n_locations, n_depots, max_vehicles, max_capacity)
data[:20]

(111, 6) (111, 111) 100 11 25 200


array([[ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 41.,  49.,  10.,  73., 204.,  10.],
       [ 35.,  17.,   7.,  18., 147.,  10.],
       [ 55.,  45.,  13.,  76., 165.,  10.],
       [ 55.,  20.,  19.,  73., 195.,  10.],
       [ 15.,  30.,  26.,  20., 167.,  10.],
       [ 25.,  30.,   3.,  49., 158.,  10.],
       [ 20.,  50.,   5.,  36., 135.,  10.],
       [ 10.,  43.,   9.,  50., 149.,  10.],
       [ 55.,  60.,  16.,  47., 156.,  10.]])

In [8]:
data

array([[ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 41.,  49.,  10.,  73., 204.,  10.],
       [ 35.,  17.,   7.,  18., 147.,  10.],
       [ 55.,  45.,  13.,  76., 165.,  10.],
       [ 55.,  20.,  19.,  73., 195.,  10.],
       [ 15.,  30.,  26.,  20., 167.,  10.],
       [ 25.,  30.,   3.,  49., 158.,  10.],
       [ 20.,  50.,   5.,  36., 135.,  10.],
       [ 10.,  43.,   9.,  50., 149.,  10.],
       [ 55.,  60.,  16.,  47., 156.,  10.],
       [ 30.,  60.,  16.,  85., 172.,  10.],
       [ 20.,  65.,  12.,  33., 152.,  10.],
       [ 5

In [9]:
assert(data.shape[0] == n_locations + n_depots and data.shape[1] == 6)
assert(graph.edges.shape[0] == n_locations + n_depots and graph.edges.shape[1] == graph.edges.shape[0])

graph.edges

array([[1.00000000e+04, 1.00000000e+04, 1.00000000e+04, ...,
        2.12602916e+01, 1.74928557e+01, 2.40416306e+01],
       [1.00000000e+04, 1.00000000e+04, 1.00000000e+04, ...,
        2.12602916e+01, 1.74928557e+01, 2.40416306e+01],
       [1.00000000e+04, 1.00000000e+04, 1.00000000e+04, ...,
        2.12602916e+01, 1.74928557e+01, 2.40416306e+01],
       ...,
       [2.12602916e+01, 2.12602916e+01, 2.12602916e+01, ...,
        1.00000000e+04, 5.09901951e+00, 3.16227766e+00],
       [1.74928557e+01, 1.74928557e+01, 1.74928557e+01, ...,
        5.09901951e+00, 1.00000000e+04, 8.24621125e+00],
       [2.40416306e+01, 2.40416306e+01, 2.40416306e+01, ...,
        3.16227766e+00, 8.24621125e+00, 1.00000000e+04]])

## Objective function

In [10]:
def maco_objective(tour, n_locations, n_depots, data, graph, max_capacity):
    
    curr_time, curr_dist, curr_load = 0, 0, 0

    for i in range(len(tour)-1):

        ni, nj = tour[i], tour[i+1]
        tij = dist([data[ni,0], data[ni,1]], [data[nj,0], data[nj,1]])
        curr_dist += tij

        # not necessary, just checking correctness
        if nj < n_depots:   # destination location is a depot
            curr_time, curr_load = 0, 0

        else:               # destination location is a client location
            curr_load += data[nj,2]
            curr_time = max(curr_time + tij, data[nj,3])
            # assert(data[nj,3] <= curr_time <= data[nj,4]), (nj, data[nj][3], data[nj][4], curr_time, tour)
            # assert(curr_load <= max_capacity), 'capacity'
            curr_time += data[nj,5]
        
    # assert(tour[0] < n_depots and tour[-1] < n_depots), 'depots'
    
    return curr_dist

## Ant Definition

In [11]:
class Ant:
    def __init__(self, tour, n_locations, n_depots, data, graph, max_capacity) -> None:
        self.tour = tour
        self.cost = maco_objective(tour, n_locations, n_depots, data, graph, max_capacity)
        self.veh = sum([x < n_depots for x in tour]) - 1 # CHECK LATER
        # clients visited
        self.visi = len(set(tour)) - (self.veh + 1) # CHECK LATER

    def __repr__(self) -> str:
        return f'Ant(tour={self.tour}, o={self.cost}, veh={self.veh}, visi={self.visi})'

    def __lt__(self, other):
        return self.cost < other.cost

## Calculating feasible solution

In [12]:
def feasible_initial_solution(n_locations, n_depots, graph, data, max_capacity):
    tour = [0]
    vis = np.zeros(n_locations + n_depots)
    vis[0] = 1
    load, curr_time = 0, 0
    next_depot = 1

    while np.sum(vis[-n_locations:]) < n_locations: # while all client locations are yet to be visited
        last = tour[-1]
        mins = np.argsort(graph.edges[last])
        # print(graph.edges[last])
        # print(mins)
        # print(tour)
        for x in mins:
            # considering only unvisited client locations
            if (x >= n_depots and not vis[x] and curr_time + graph.edges[last,x] <= data[x,4]
                and load + data[x, 2] <= max_capacity):
                curr_time = max(curr_time + graph.edges[last][x], data[x, 3])
                curr_time += data[x,5]
                load += data[x, 2]
                tour.append(x)
                vis[x] = 1
                break
        else:
            # if no valid unvisited client locations, return to depot and start new cycle
            curr_time = 0
            load = 0
            tour.append(next_depot)
            vis[next_depot] = 1
            next_depot += 1 
    if tour[-1] >= n_depots:
        tour.append(next_depot)
    sol = Ant(tour, n_locations, n_depots, data, graph, max_capacity)
    return sol

n_vehicles = 30
data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
psi_fis = feasible_initial_solution(n_locations, n_depots, graph, data, max_capacity)
print(psi_fis)

Ant(tour=[0, 83, 88, 70, 51, 103, 102, 104, 86, 34, 1, 57, 58, 56, 42, 110, 98, 107, 33, 80, 31, 2, 119, 36, 124, 125, 127, 122, 89, 123, 67, 130, 3, 43, 117, 32, 87, 45, 73, 44, 4, 82, 48, 90, 113, 114, 35, 129, 126, 121, 5, 99, 61, 118, 92, 40, 120, 62, 93, 94, 6, 106, 109, 108, 64, 111, 63, 81, 39, 65, 7, 100, 60, 50, 96, 101, 95, 8, 37, 112, 78, 77, 66, 79, 49, 76, 9, 128, 115, 91, 46, 74, 116, 47, 75, 10, 84, 85, 55, 69, 54, 11, 72, 71, 52, 105, 53, 97, 12, 38, 41, 59, 13, 68, 14], o=1370.2311363260492, veh=14, visi=100)


In [13]:
def infeasible_solution(n_locations, n_depots, n_vehicles, graph, data, max_capacity):
    tour = [0]
    vis = np.zeros(n_locations + n_depots)
    vis[0] = 1
    load, curr_time = 0, 0
    next_depot = 1

    while np.sum(vis[-n_locations:]) < n_locations: # while all client locations are yet to be visited
        last = tour[-1]
        mins = np.argsort(graph.edges[last])
        
        for x in mins:
            # considering only unvisited client locations
            if (x >= n_depots and not vis[x] and curr_time + graph.edges[last,x] <= data[x,4]
                and load + data[x, 2] <= max_capacity):
                curr_time = max(curr_time + graph.edges[last][x], data[x, 3])
                curr_time += data[x,5]
                load += data[x, 2]
                tour.append(x)
                vis[x] = 1
                break
        else:
            # if no valid unvisited client locations, return to depot and start new cycle
            curr_time = 0
            load = 0
            tour.append(next_depot)
            vis[next_depot] = 1
            next_depot += 1 
            if next_depot == n_depots:
                break
    if tour[-1] >= n_depots:
        tour.append(next_depot)
    sol = Ant(tour, n_locations, n_depots, data, graph, max_capacity)
    return sol

n_vehicles = 10
data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
psi_is = infeasible_solution(n_locations, n_depots, n_vehicles, graph, data, max_capacity)
print(psi_is)

Ant(tour=[0, 63, 68, 50, 31, 83, 82, 84, 66, 14, 1, 37, 38, 36, 22, 90, 78, 87, 13, 60, 11, 2, 99, 16, 104, 105, 107, 102, 69, 103, 47, 110, 3, 23, 97, 12, 67, 25, 53, 24, 4, 62, 28, 70, 93, 94, 15, 109, 106, 101, 5, 79, 41, 98, 72, 20, 100, 73, 42, 30, 76, 6, 86, 89, 43, 91, 19, 61, 40, 80, 58, 7, 17, 92, 18, 55, 27, 71, 26, 56, 8, 108, 95, 54, 96, 48, 52, 49, 9, 64, 65, 35, 34, 10], o=962.1513591870481, veh=10, visi=83)


In [14]:
def valid_tour(tour, n_locations, n_depots, data, graph, max_capacity):
    
    curr_time, curr_dist, curr_load = 0, 0, 0

    for i in range(len(tour)-1):

        ni, nj = tour[i], tour[i+1]
        tij = dist([data[ni,0], data[ni,1]], [data[nj,0], data[nj,1]])
        curr_dist += tij

        # not necessary, just checking correctness
        if nj < n_depots:   # destination location is a depot
            curr_time, curr_load = 0, 0

        else:               # destination location is a client location
            curr_load += data[nj,2]
            curr_time = max(curr_time + tij, data[nj,3])
            if not (data[nj,3] <= curr_time <= data[nj,4]):
                return False
            if not(curr_load <= max_capacity):
                return False
            curr_time += data[nj,5]
        
    if not (tour[0] < n_depots and tour[-1] < n_depots):
        return False    
        
    return True

## Local Search Functions

In [15]:
def split_into_routes(tour, n_vehicles):
    routes = []
    cur_route = [tour[0]]
    for x in tour[1:]:
        if x <= n_vehicles:
            cur_route.append(x)
            routes.append(cur_route)
            cur_route = [x]
        else:
            cur_route.append(x)
    return routes

In [16]:
def merge_into_tour(routes, n_vehicles):
    tour = [0]
    for route in routes:
        for x in route[1:]:
            tour.append(x)
    dep = 1
    for i in range(1, len(tour)):
        if tour[i] == 0:
            tour[i] = dep
            dep += 1
    return tour

In [17]:
import itertools

def two_opt(a, i, j):
    if i == 0:
        return a[j:i:-1] + [a[i]] + a[j + 1:]
    return a[:i] + a[j:i - 1:-1] + a[j + 1:]


def cross(a, b, i, j):
    return a[:i] + b[j:], b[:j] + a[i:]


def insertion(a, b, i, j):
    # print(a, b, i, j)
    if len(a) == 0:
        return a, b
    while i >= len(a):
        i -= len(a)
    return a[:i] + a[i + 1:], b[:j] + [a[i]] + b[j:]


def swap(a, b, i, j):
    # print(a, b, i, j)
    if i >= len(a) or j >= len(b):
        return a, b
    a, b = a.copy(), b.copy()
    a[i], b[j] = b[j], a[i]
    return a, b

In [18]:
def valid_route_distance(route, data, graph, max_capacity):
    
    curr_time, curr_dist, curr_load = 0, 0, 0

    for i in range(len(route)-1):

        ni, nj = route[i], route[i+1]
        tij = dist([data[ni,0], data[ni,1]], [data[nj,0], data[nj,1]])
        curr_dist += tij
        curr_load += data[nj,2]
        curr_time = max(curr_time + tij, data[nj,3])
        if not (data[nj,3] <= curr_time <= data[nj,4]):
            return False, 10000
        if not(curr_load <= max_capacity):
            return False, 10000
        curr_time += data[nj,5]
        
    return True, curr_dist

In [19]:
def local_search(tour, n_vehicles, data, graph, max_capacity):
    routes = split_into_routes(tour, n_vehicles)

    can_impr = True
    while can_impr:
        can_impr = False
        
        for i, j in itertools.combinations(range(len(routes)), 2):

            vri, dri = valid_route_distance(routes[i], data, graph, max_capacity)
            vrj, drj = valid_route_distance(routes[j], data, graph, max_capacity)

            for k, l in itertools.product(range(len(routes[i])), range(len(routes[j]))):
                for func in [cross, insertion, swap]:
                    c1, c2 = func(routes[i][1:-1], routes[j][1:-1], k, l)
                    n1, n2 = [0] + c1 + [0], [0] + c2 + [0]

                    vn1, dn1 = valid_route_distance(n1, data, graph, max_capacity)
                    vn2, dn2 = valid_route_distance(n2, data, graph, max_capacity)

                    if vn1 and vn2:
                        if dn1 + dn2 < dri + drj:
                            routes[i] = n1
                            routes[j] = n2
                            can_impr = True
                            
    return merge_into_tour(routes, n_vehicles)

## Create new ant

In [20]:
def new_active_ant(local, incom, n_locations, n_depots, n_vehicles, data, graph, tau, tau0, beta, max_capacity, rho):

    # print(data.shape, graph.edges.shape)
    inc = incom.copy()

    tour = [np.random.choice(n_depots)]
    curr_time, curr_load = 0, 0

    vis = np.zeros(n_locations + n_depots)
    vis[tour[0]] = 1
    eta = np.full(n_locations + n_depots, 0.00001)

    # first insertion of nodes by ACO
    feasible_node = True
    while feasible_node:

        feasible_node = False
        last = tour[-1]
        for x in range(n_locations + n_depots):
            if (not vis[x] and curr_time + graph.edges[last,x] <= data[x,4]
                    and curr_load + data[x, 2] <= max_capacity):
                feasible_node = True
                deliv_time = max(curr_time + graph.edges[last,x], data[x, 3])
                delta_time = deliv_time - curr_time
                distance = delta_time * (data[x,4] - curr_time)
                # print(distance)
                distance = max(1, distance - inc[x])
                eta[x] = 1 / distance
            else:
                eta[x] = 0

        if not feasible_node:
            break

        probs = (tau[last]) * (eta ** beta)
        probs = probs / np.sum(probs)
        
        # next = np.random.choice(n_locations + n_depots, p=probs)
        q = np.random.rand()
        if q < 0.9:
            next = np.argmax(probs)
        else:
            probs[probs != 0.0] = 1
            probs = probs / np.sum(probs)
            next = np.random.choice(n_locations + n_depots, p=probs)

        tour.append(next)
        vis[next] = 1

        curr_time = max(curr_time + graph.edges[last][x], data[next, 3])
        curr_time += data[next,5]

        curr_load += data[next, 2]
        # print('A', tour)

        tau[last][next] = (1 - rho) * tau[last][next] + rho * tau0
    
    # print('B', tour)

    # there still may remain some non inserted nodes
    for x in range(n_locations + n_depots):
        if not vis[x]:
            for g in range(1, len(tour) + 1):
                tc = tour.copy()
                tc.insert(g, x)
                if valid_tour(tc, n_locations, n_depots, data, graph, max_capacity):
                    tour = tc
                    vis[x] = 1
                    break

    if local:
        tour = local_search(tour, n_vehicles, data, graph, max_capacity)

    # assert(len(tour) == n_locations)

    sol = Ant(tour, n_locations, n_depots, data, graph, max_capacity)
    return sol

In [21]:
# n_vehicles = 10
# data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
# incom = np.zeros(n_locations + n_depots)
# tau0 = 10/np.sum(graph.edges)
# tau = np.full(graph.edges.shape, tau0)
# beta = 1
# rho = 0.25
# local = False
# new_active_ant(local, incom, n_locations, n_depots, n_vehicles, data, graph, tau, tau0, beta, max_capacity, rho)

## ACO Vehicle function

In [22]:
# def aco_vehicle(n_vehicles, n_ants, psi_gb, beta, rho):
def aco_vehicle(n_vehicles, n_ants, beta, rho):
    global psi_gb

    n_depots = n_vehicles + 1
    data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
    tau0 = 10/np.sum(graph.edges)
    tau = np.full(graph.edges.shape, tau0)

    # initial solution, may be infeasible
    psi_vei = infeasible_solution(n_locations, n_depots, n_vehicles, graph, data, max_capacity)
    # print(psi_vei)

    incom = np.zeros(n_locations + n_depots)
    for i in tqdm(range(10)):
        ants = [None for i in range(n_ants)]

        for i in range(n_ants):
            local = False
            ants[i] = new_active_ant(local, incom, n_locations, n_depots, 
                                    n_vehicles, data, graph, tau, tau0, 
                                    beta, max_capacity, rho)
            for x in ants[i].tour:
                incom[x] += 1
 
        for i in range(n_ants):
            if ants[i].visi > psi_vei.visi:
                psi_vei = copy.deepcopy(ants[i])
                incom = np.zeros(n_locations + n_depots)
                if ants[i].visi == n_locations:
                    psi_gb = copy.deepcopy(ants[i])
                    return psi_gb
            
        # updating tau values
        tau = (1-rho) * tau
        for j in range(len(psi_vei.tour) - 1):
            cur, nxt = psi_vei.tour[j:j+2]
            tau[cur][nxt] += 1/psi_vei.cost
            tau[nxt][cur] += 1/psi_vei.cost
        for j in range(len(psi_gb.tour) - 1):
            cur, nxt = psi_gb.tour[j:j+2]
            if cur < tau.shape[0] and nxt < tau.shape[0]:
                tau[cur][nxt] += 1/psi_gb.cost
                tau[nxt][cur] += 1/psi_gb.cost

    return psi_vei

In [23]:
# n_vehicles = 17
# beta = 1
# rho = 0.25
# n_ants = 20

# aco_vehicle(n_vehicles, n_ants, psi_fis, beta, rho)

## ACO Time function

In [24]:
# def aco_time(n_vehicles, n_ants, psi_gb, beta, rho):
def aco_time(n_vehicles, n_ants, beta, rho):
    global psi_gb

    # psi_gb.cost = 10000
    n_depots = n_vehicles + 1
    data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
    tau0 = 10/np.sum(graph.edges)
    tau = np.full(graph.edges.shape, tau0)

    incom = np.zeros(n_locations + n_depots)
    for i in tqdm(range(10)):
        ants = [None for i in range(n_ants)]

        for i in range(n_ants):
            local = True
            ants[i] = new_active_ant(local, incom, n_locations, n_depots, 
                                    n_vehicles, data, graph, tau, tau0, 
                                    beta, max_capacity, rho)
 
        for i in range(n_ants):
            # print(len(ants[i].tour), ants[i].cost)
            if len(ants[i].tour) == n_locations + n_depots and ants[i].cost < psi_gb.cost:
                psi_gb = ants[i]
            
        # updating tau values
        tau = (1-rho) * tau
        # print(psi_gb)
        for j in range(len(psi_gb.tour) - 1):
            cur, nxt = psi_gb.tour[j:j+2]
            if cur < tau.shape[0] and nxt < tau.shape[0]:
                tau[cur][nxt] += 1/psi_gb.cost
                tau[nxt][cur] += 1/psi_gb.cost

    return psi_gb

In [25]:
# n_vehicles = 19
# beta = 1
# rho = 0.25
# n_ants = 20

# aco_time(n_vehicles, n_ants, psi_fis, beta, rho)

## Higher level multiple ACO function

In [26]:
def maco_vrptw(n_ants, beta, rho):
    
    veh = psi_gb.veh
    for i in range(30):

        for i in range(5):
            psi1 = aco_vehicle(veh - 1, n_ants, beta, rho)
            print('\nvehicle => ', psi1)
            if psi1.veh <= veh and psi1.visi == 100:
                veh = psi1.veh
                break
        else:
            psi2 = aco_time(veh, n_ants, beta, rho)
            print('\ntime => ', psi2)
            break
        
    return psi_gb

In [27]:
n_vehicles = 30
data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
psi_gb = feasible_initial_solution(n_locations, n_depots, graph, data, max_capacity)

n_ants = 10
beta = 1
rho = 0.1
best_tour = maco_vrptw(n_ants, beta, rho)
best_tour

  0%|          | 0/10 [00:00<?, ?it/s]
vehicle =>  Ant(tour=[0, 66, 71, 53, 34, 86, 85, 87, 69, 17, 1, 40, 41, 39, 25, 93, 81, 90, 16, 63, 14, 2, 102, 19, 107, 108, 110, 105, 72, 106, 50, 113, 3, 26, 100, 15, 70, 28, 56, 27, 4, 65, 31, 73, 96, 97, 18, 112, 109, 104, 5, 82, 44, 101, 75, 23, 103, 76, 45, 33, 79, 6, 89, 92, 91, 47, 94, 46, 64, 22, 48, 7, 83, 43, 84, 78, 37, 67, 8, 20, 95, 61, 60, 49, 62, 32, 59, 9, 111, 98, 74, 29, 57, 99, 30, 58, 10, 55, 54, 35, 88, 36, 52, 38, 68, 11, 21, 24, 77, 12, 42, 80, 51, 13], o=1352.7622316883844, veh=13, visi=100)
  0%|          | 0/10 [00:00<?, ?it/s]
vehicle =>  Ant(tour=[0, 65, 70, 52, 33, 85, 84, 86, 68, 16, 1, 39, 40, 38, 24, 92, 80, 89, 15, 62, 13, 2, 101, 18, 106, 107, 109, 104, 71, 105, 49, 112, 3, 25, 99, 14, 69, 27, 55, 26, 4, 64, 30, 72, 95, 96, 17, 111, 108, 103, 5, 81, 43, 100, 19, 94, 60, 59, 48, 61, 58, 6, 88, 91, 90, 46, 93, 45, 63, 21, 47, 7, 82, 42, 32, 78, 83, 77, 44, 8, 110, 97, 73, 28, 56, 98, 29, 57, 9, 66, 67, 37, 51, 36,

Ant(tour=[0, 83, 88, 70, 51, 103, 102, 104, 86, 34, 1, 57, 58, 56, 42, 110, 98, 107, 33, 80, 31, 2, 119, 36, 124, 125, 127, 122, 89, 123, 67, 130, 3, 43, 117, 32, 87, 45, 73, 44, 4, 82, 48, 90, 113, 114, 35, 129, 126, 121, 5, 99, 61, 118, 92, 40, 120, 62, 93, 94, 6, 106, 109, 108, 64, 111, 63, 81, 39, 65, 7, 100, 60, 50, 96, 101, 95, 8, 37, 112, 78, 77, 66, 79, 49, 76, 9, 128, 115, 91, 46, 74, 116, 47, 75, 10, 84, 85, 55, 69, 54, 11, 72, 71, 52, 105, 53, 97, 12, 38, 41, 59, 13, 68, 14], o=1370.2311363260492, veh=14, visi=100)